### Database works.
In this **notebook** we have a lot of good stuff:
* Connecting to a database
* Creating schema, table
* Renaming files
* Import raster data to this table
* Organize the table, addi

In [1]:
%load_ext sql
import os, getpass, time
import xuleta as xu

In [2]:
user = input("Username: ")
password = getpass.getpass('Enter your password: ')
database = input("Database: ")
connection_string = "postgresql://%s:%s@localhost/%s" %(user,password,database)
%sql $connection_string

Username: denis
Enter your password: ········
Database: drought


'Connected: denis@drought'

#### Renaming files using xuleta

In [ ]:
folder = "/media/denis/seagate/MODIS/R_MYD17A2H/h11v04/Gross_PP_8Days_500m_v6/GPP"
xu.renamedate(folder=folder,oldf="%Y_%j",newf="%Y-%m-%d",wts=14)

---

---
## Creating Schemas, tables
**Working on MODIS TILE h11v04 Variable GPP**

In [ ]:
%sql CREATE SCHEMA modis11v04;

In [ ]:
%%time
%%sql CREATE TABLE modis11v04.gpp(
        rid serial NOT NULL,
        rast raster,
        filename text,
        acquisition_range daterange,
        CONSTRAINT modis11v04_gpp_pkey
        PRIMARY KEY (rid));
        CREATE INDEX modis11v04_gpp_wkb_rast_idx
        ON modis11v04.gpp
        USING GIST (ST_ConvexHull(rast));
        COMMENT ON TABLE modis11v04.gpp
        IS 'Table that stores values of GPP - 8day for tile h11v04.';

### Importing images to the database
Make sure the schema.table matches the images I'm importing.

You can watch the verbose in the terminal to make sure that things are going ok.

In [ ]:
imfolder = "/media/denis/seagate/MODIS/R_MYD17A2H/h11v04/Gross_PP_8Days_500m_v6/GPP/"

t0 = time.time()
os.system("raster2pgsql -a -C -F -M -s 4326 -t 200x200 -N 32767 "+imfolder+
#change the schema.table
          "*.tif modis11v04.gpp | psql -p 5432 -d drought -U denis")

taime = time.time() - t0
xu.shemale(taime/60)

You can confirm that the raster was properly loaded with all its attributes by looking at the raster_columns view, which stores raster metadata (here, you only retrieve the table’s schema, name, SRID and NoData value, but it is a good practice to examine all information stored in this view)

In [ ]:
%%time
%%sql SELECT
    r_table_schema AS schema,
    r_table_name AS table,
    srid,
    nodata_values AS nodata
    FROM raster_columns
    WHERE r_table_name = 'modis11v04.gpp';

In [ ]:
#%lsmagic

In the case of more complex filenames with a variable number of characters, you could still retrieve the encoded date using the substring function, by extracting the relevant characters relative to some other characters found first using the position function. Let us now update the table by converting the filenames into the date ranges according to the convention used in file naming (note that there is an
additional constraint that selects 1 January when the start date ? 16 days exceeds the beginning of the year):
### atente para o 8 ou 16 dias

In [ ]:
%%time
%%sql    UPDATE modis11v04.gpp
    SET acquisition_range = daterange(
    (substring(filename FROM 1 FOR 4) || '-' ||
    substring(filename FROM 6 FOR 2) || '-' ||
    substring(filename FROM 9 FOR 2))::date,
    LEAST((substring(filename FROM 1 FOR 4) || '-' ||
    substring(filename FROM 6 FOR 2) || '-' ||
    substring(filename FROM 9 FOR 2))::date + 8,
    (substring(filename FROM 1 FOR 4)::integer + 1
    || '-' || '01' || '-' || '01')::date));

As for any type of column, if the table contains a large number of rows
(e.g. [10,000), querying based on the acquisition_range will be faster if you first index it (you can do it even if the table is not that big, as the PostgreSQL planner will determine whether the query will be faster by using the index or not):

In [ ]:
%%time
%%sql     CREATE INDEX gpp_acquisition_range_idx
    ON modis11v04.gpp (acquisition_range);

Now, each tile (and therefore each pixel) has a spatial and a temporal component and thus can be queried according to both criteria. For instance, these are the 10 first tiles corresponding to 1 March 2008, using the ‘@[’ operator (‘contains’). Note that this is a leap year so that the corresponding period ends on 5 March:

In [ ]:
%%time
%%sql     SELECT rid, filename, acquisition_range
    FROM modis11v04.gpp
    WHERE acquisition_range @> '2015-12-01'::date
    LIMIT 5;

** Retrieving time-series from a given point**

using this query, we can get the time-series for a point, e.g. -110.868,45.050. It's super fast!

In [4]:
%%time
%%sql SELECT 
        rid, 
        acquisition_range,
        ST_Value(rast, ST_SetSRID(ST_MakePoint(-92.891,42.436), 4326)) AS gpp
    FROM modis11v04.gpp
         WHERE ST_Intersects(ST_SetSRID(ST_MakePoint(-92.891,42.436), 4326), rast)
           AND acquisition_range && '[2002-01-01,2016-12-31]'::daterange
    ORDER BY acquisition_range;

660 rows affected.
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 675 ms


rid,acquisition_range,gpp
290,"DateRange(datetime.date(2002, 7, 28), datetime.date(2002, 8, 5), '[)')",517.0
698,"DateRange(datetime.date(2002, 8, 5), datetime.date(2002, 8, 13), '[)')",434.0
1106,"DateRange(datetime.date(2002, 8, 13), datetime.date(2002, 8, 21), '[)')",593.0
1514,"DateRange(datetime.date(2002, 8, 21), datetime.date(2002, 8, 29), '[)')",542.0
1922,"DateRange(datetime.date(2002, 8, 29), datetime.date(2002, 9, 6), '[)')",450.0
2330,"DateRange(datetime.date(2002, 9, 6), datetime.date(2002, 9, 14), '[)')",365.0
2738,"DateRange(datetime.date(2002, 9, 14), datetime.date(2002, 9, 22), '[)')",300.0
3146,"DateRange(datetime.date(2002, 9, 22), datetime.date(2002, 9, 30), '[)')",259.0
3554,"DateRange(datetime.date(2002, 9, 30), datetime.date(2002, 10, 8), '[)')",78.0
3962,"DateRange(datetime.date(2002, 10, 8), datetime.date(2002, 10, 16), '[)')",108.0


**NEXT STEP**

Implement the query using a shapefile. That is the goal! This one here retrieves all the raster values withing a poligon in an array. From this we could to the stats externally, but we don't need it.

In [ ]:
%%time
%%sql SELECT gid, acquisition_range, (st_dumpvalues(ST_Clip(rast,geom, true))).valarray
    FROM
    modis10v04.gpp , public.shape10v04
    WHERE st_intersects(rast,geom)
        AND acquisition_range && '[2015-01-01,2016-12-31]'::daterange 
        AND gid = 1 
    ORDER BY acquisition_range;

#### AND I FINALLY got what I WANT

This query retrieves all the raster stats by polygon. 

In [ ]:
%%time
%%sql SELECT id, acquisition_range, (st_SummaryStats(ST_Clip(rast,1,geom, true))).mean as media,
    (st_SummaryStats(ST_Clip(rast,1,geom, true))).sum as soma,
    (st_SummaryStats(ST_Clip(rast,1,geom, true))).count as pixels,
    (st_SummaryStats(ST_Clip(rast,1,geom, true))).stddev as desvio,
    power((st_SummaryStats(ST_Clip(rast,1,geom, true))).stddev,2) as variancia,
    (st_SummaryStats(ST_Clip(rast,1,geom, true))).max as maximo
FROM modis10v04.gpp , public.shape10v04
WHERE st_intersects(rast,geom)
    AND acquisition_range && '[2005-01-01,2016-12-31]'::daterange 
    AND id = 1 
ORDER BY acquisition_range;

To put the result in a pandas dataframe we have to use single-line %sql statement, which make thinks a bit ugly, but it works!

In [ ]:
result = %sql SELECT id, acquisition_range, (st_SummaryStats(ST_Clip(rast,1,geom, true))).mean as media FROM modis10v04.gpp , public.shape10v04 WHERE st_intersects(rast,geom) AND acquisition_range && '[2005-01-01,2016-12-31]'::daterange AND id =1 ORDER BY acquisition_range;

In [ ]:
result.DataFrame()

In [ ]:
result.csv(filename='work.csv')